<h3> Polynomial Regression </h3>

To add polynomial features to the data, we use PolynomialFeatures - remember to set the polynomial degree

For example, to add 2nd-degree features

In [1]:
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
X = np.array([[0, 1],
              [2, 3],
              [4, 5]])

poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

X_poly

array([[ 0.,  1.,  0.,  0.,  1.],
       [ 2.,  3.,  4.,  6.,  9.],
       [ 4.,  5., 16., 20., 25.]])

Or 3rd-degree features

In [6]:
poly_3 = PolynomialFeatures(degree=3, include_bias=False)

X_poly_3 = poly_3.fit_transform(X)

X_poly_3

array([[  0.,   1.,   0.,   0.,   1.,   0.,   0.,   0.,   1.],
       [  2.,   3.,   4.,   6.,   9.,   8.,  12.,  18.,  27.],
       [  4.,   5.,  16.,  20.,  25.,  64.,  80., 100., 125.]])

<h3>Adding Polynomial Features in a Pipeline</h3>

In a real analysis, we can add the new features as another step in our pipeline, for example, in the auto-mpg data

In [4]:
import pandas as pd

auto = pd.read_csv('auto-mpg.csv')

from sklearn.model_selection import ShuffleSplit

split = ShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

for train_index, test_index in split.split(auto):
    train_set = auto.loc[train_index]
    test_set = auto.loc[test_index]
    
trainX = train_set.drop('mpg',axis=1)
trainY = train_set['mpg']
testX = test_set.drop('mpg',axis=1)
testY = test_set['mpg']

trainX.shape, testX.shape, trainY.shape, testY.shape

((298, 7), (100, 7), (298,), (100,))

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

#pipeline for numeric features
#we need to impute horsepower
num_cols = trainX.columns[:-1] #because the last column is class
num_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('poly', PolynomialFeatures(degree=2)),
    ('standardize', StandardScaler())
])

#pipeline for class features
cat_cols = trainX.columns[-1:] #because the last column is class
cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder())
])

#full pipeline - combine numeric and class pipelines
full_pipeline = ColumnTransformer([
    ('numeric', num_pipeline, num_cols),
    ('class', cat_pipeline, cat_cols)
])

In [37]:
trainX_poly2 = full_pipeline.fit_transform(trainX)
trainX_poly2.shape

(298, 31)

After adding polynomial features, we fit a linear regression model like normal

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

poly_reg = LinearRegression()
poly_reg.fit(trainX_poly2,trainY)

mse_lr_cv = - cross_val_score(poly_reg, trainX_poly2, trainY, cv=5, scoring='neg_mean_squared_error')
r2_lr_cv = cross_val_score(poly_reg, trainX_poly2, trainY, cv=5, scoring='r2')

print('Poly Reg 2nd-degree Train R2: ', poly_reg.score(trainX_poly2,trainY))
print('Poly Reg 2nd-degree CV MSE: ', mse_lr_cv.mean())
print('Poly Reg 2nd-degree CV R2: ', r2_lr_cv.mean())

Poly Reg 2nd-degree Train R2:  0.8000350958113276
Poly Reg 2nd-degree CV MSE:  17.706947104069584
Poly Reg 2nd-degree CV R2:  0.7064347089744643


As you can see, the training R2 is much better than the cross-validation R2. What does this means?

It turns out, adding polynomial features is a very easy way to overfit the training data; the higher the degree, the easier it gets to overfit.

Let's see a 3rd-degree polynomial model

In [35]:
num_pipeline_2 = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('poly', PolynomialFeatures(degree=3)),
    ('standardize', StandardScaler())
])

#full pipeline - combine numeric and class pipelines
full_pipeline_2 = ColumnTransformer([
    ('numeric', num_pipeline_2, num_cols),
    ('class', cat_pipeline, cat_cols)
])

trainX_poly3 = full_pipeline_2.fit_transform(trainX)
trainX_poly3.shape

(298, 87)

In [18]:
poly_reg.fit(trainX_poly3,trainY)

mse_lr_cv = - cross_val_score(poly_reg, trainX_poly3, trainY, cv=5, scoring='neg_mean_squared_error')
r2_lr_cv = cross_val_score(poly_reg, trainX_poly3, trainY, cv=5, scoring='r2')

print('Poly Reg 3rd-degree Train R2: ', poly_reg.score(trainX_poly3,trainY))
print('Poly Reg 3rd-degree CV MSE: ', mse_lr_cv.mean())
print('Poly Reg 3rd-degree CV R2: ', r2_lr_cv.mean())

Poly Reg 3rd-degree Train R2:  0.8482560536976587
Poly Reg 3rd-degree CV MSE:  85.82578213135113
Poly Reg 3rd-degree CV R2:  -0.41270046586832


So, training R2 is even higher than before, and cross-validation R2 is negative now, all of which suggests this model is overfitting the data even more severely. 

Let's try a regular linear model without polynomial features

In [38]:
num_pipeline_3 = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('standardize', StandardScaler())
])

#full pipeline - combine numeric and class pipelines
full_pipeline_3 = ColumnTransformer([
    ('numeric', num_pipeline_3, num_cols),
    ('class', cat_pipeline, cat_cols)
])

trainX_ln = full_pipeline_3.fit_transform(trainX)
trainX_ln.shape

(298, 9)

In [23]:
linear_reg = LinearRegression()
linear_reg.fit(trainX_ln,trainY)

mse_lr_cv = - cross_val_score(poly_reg, trainX_ln, trainY, cv=5, scoring='neg_mean_squared_error')
r2_lr_cv = cross_val_score(poly_reg, trainX_ln, trainY, cv=5, scoring='r2')

print('Linear Reg Train R2: ', linear_reg.score(trainX_ln,trainY))
print('Linear Reg CV MSE: ', mse_lr_cv.mean())
print('Linear Reg CV R2: ', r2_lr_cv.mean())

Linear Reg Train R2:  0.8172751189834604
Linear Reg CV MSE:  12.138420248599349
Linear Reg CV R2:  0.8030148839899887


<h3>Ridge Regression</h3>

We use the Ridge class, also from the sklearn.linear_model module. A default Ridge regression model has alpha=1.

Let's try the model on the 2nd-degree polynomial data

In [17]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge()

ridge_reg.fit(trainX_poly2,trainY)

mse_lr_cv = - cross_val_score(ridge_reg, trainX_poly2, trainY, cv=5, scoring='neg_mean_squared_error')
r2_lr_cv = cross_val_score(ridge_reg, trainX_poly2, trainY, cv=5, scoring='r2')

print('Poly Reg Train R2: ', ridge_reg.score(trainX_poly2,trainY))
print('Poly Reg CV MSE: ', mse_lr_cv.mean())
print('Poly Reg CV R2: ', r2_lr_cv.mean())

Poly Reg Train R2:  0.8713112281784805
Poly Reg CV MSE:  9.18003939637259
Poly Reg CV R2:  0.8507888466222286


You can see the immediate improvement, the model no longer overfits, and even gets better than all previous linear models. However, adding more features may still make the model worse, for example, in the 3rd-degree polynomial data

In [24]:
ridge_reg.fit(trainX_poly3,trainY)

mse_lr_cv = - cross_val_score(ridge_reg, trainX_poly3, trainY, cv=5, scoring='neg_mean_squared_error')
r2_lr_cv = cross_val_score(ridge_reg, trainX_poly3, trainY, cv=5, scoring='r2')

print('Poly Reg Train R2: ', ridge_reg.score(trainX_poly3,trainY))
print('Poly Reg CV MSE: ', mse_lr_cv.mean())
print('Poly Reg CV R2: ', r2_lr_cv.mean())

Poly Reg Train R2:  0.8810875212915859
Poly Reg CV MSE:  9.33117703231586
Poly Reg CV R2:  0.8479611655203125


And in the linear data, we have very similar result to OLS model

In [25]:
ridge_reg = Ridge()

ridge_reg.fit(trainX_ln,trainY)

mse_lr_cv = - cross_val_score(ridge_reg, trainX_ln, trainY, cv=5, scoring='neg_mean_squared_error')
r2_lr_cv = cross_val_score(ridge_reg, trainX_ln, trainY, cv=5, scoring='r2')

print('Poly Reg Train R2: ', ridge_reg.score(trainX_ln,trainY))
print('Poly Reg CV MSE: ', mse_lr_cv.mean())
print('Poly Reg CV R2: ', r2_lr_cv.mean())

Poly Reg Train R2:  0.8171790286982343
Poly Reg CV MSE:  12.107294352932252
Poly Reg CV R2:  0.803609988247809


As mention, alpha is a hyper-parameter you need to finetune. We can use grid-search

In [66]:
from sklearn.model_selection import GridSearchCV

param_grid = [{'alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1 , 5, 10, 50, 100]}]

grid_search = GridSearchCV(ridge_reg, param_grid, cv=5, scoring='r2', return_train_score=True)

grid_search.fit(trainX_poly2,trainY)

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid=[{'alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5,
                                    10, 50, 100]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='r2', verbose=0)

The best alpha is

In [67]:
grid_search.best_params_

{'alpha': 0.05}

In [68]:
grid_search.best_score_

0.8521680350488623

And we can see the R2 for each value of alpha

In [44]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.8480419603031771 {'alpha': 0.001}
0.8510845383633667 {'alpha': 0.005}
0.8514615136150165 {'alpha': 0.01}
0.8521680350488623 {'alpha': 0.05}
0.8520849294753257 {'alpha': 0.1}
0.8511228624260276 {'alpha': 0.5}
0.8507888466222286 {'alpha': 1}
0.8488728987652407 {'alpha': 5}
0.845693335415354 {'alpha': 10}
0.8271070063960442 {'alpha': 50}
0.8104032968789869 {'alpha': 100}


While alpha=0.05 offers the best R2, it is not much higher than other values <= 1. However, if your project focuses on prediction performance, you may want as high R2 as possible.

We can get the best model by using best_estimator_

In [40]:
best_ridge_gs = grid_search.best_estimator_

To try the best Ridge model on the test data

In [41]:
#remember, when use pipeline to transform testing data, we don't use fit_transform() any more
testX_poly2 = full_pipeline.transform(testX)

best_ridge_gs.score(testX_poly2, testY)

0.89303597884887

And we obtain very high R2 in testing data :)

<h3>LASSO</h3>

Is very similar to Ridge regression. We use the Lasso class from linear_model module. The default alpha is also 1. 2nd-degree polynomial data seems to be the best feature set, so I'll just consider that set from now

In [48]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso()

lasso_reg.fit(trainX_poly2,trainY)

mse_lr_cv = - cross_val_score(lasso_reg, trainX_poly2, trainY, cv=5, scoring='neg_mean_squared_error')
r2_lr_cv = cross_val_score(lasso_reg, trainX_poly2, trainY, cv=5, scoring='r2')

print('Poly Reg Train R2: ', lasso_reg.score(trainX_poly2,trainY))
print('Poly Reg CV MSE: ', mse_lr_cv.mean())
print('Poly Reg CV R2: ', r2_lr_cv.mean())

Poly Reg Train R2:  0.7837047482136399
Poly Reg CV MSE:  13.737136011228907
Poly Reg CV R2:  0.778124184302442


Not very impressive performance, let's try finetuning the model

In [54]:
#lasso model are trained iteratively, so you also have to choose the maximum number of iterations with max_iter
#if you see a warning about convergence (like showing below)
#you may increase max_iter
#however it is not always necessary
lasso_reg = Lasso(max_iter=5000)

param_grid = [{'alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1 , 5, 10, 50, 100]}]

grid_search = GridSearchCV(lasso_reg, param_grid, cv=5, scoring='r2', return_train_score=True)

grid_search.fit(trainX_poly2,trainY)

c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 549.1439828607947, tolerance: 1.5077993949579833
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 412.5050948913396, tolerance: 1.5381773487394963
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 494.34983721949607, tolerance: 1.3696142184873954
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 563.8567615775291, tolerance: 1.4991293221757322
  p

GridSearchCV(cv=5, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=5000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5,
                                    10, 50, 100]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='r2', verbose=0)

The R2 of all alpha values:

In [55]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.8490242123571088 {'alpha': 0.001}
0.8508598661674691 {'alpha': 0.005}
0.8508822626025525 {'alpha': 0.01}
0.8450617682283834 {'alpha': 0.05}
0.8285816864055413 {'alpha': 0.1}
0.8023405941693722 {'alpha': 0.5}
0.778124184302442 {'alpha': 1}
0.2655134364946863 {'alpha': 5}
-0.01511192028637649 {'alpha': 10}
-0.01511192028637649 {'alpha': 50}
-0.01511192028637649 {'alpha': 100}


And the best alpha value:

In [56]:
grid_search.best_params_

{'alpha': 0.01}

Similarly, we can obtain best model with best_estimator_

In [57]:
best_lasso = grid_search.best_estimator_

And test it on the test set

In [58]:
best_lasso.score(testX_poly2, testY)

0.8952667599256074

And the result is very similar to Ridge regression

<h3>Elastic Net</h3>

The default alpha is 1, and rho is 0.5

In [60]:
from sklearn.linear_model import ElasticNet

enet = ElasticNet()

enet.fit(trainX_poly2,trainY)

mse_lr_cv = - cross_val_score(enet, trainX_poly2, trainY, cv=5, scoring='neg_mean_squared_error')
r2_lr_cv = cross_val_score(enet, trainX_poly2, trainY, cv=5, scoring='r2')

print('Poly Reg Train R2: ', enet.score(trainX_poly2,trainY))
print('Poly Reg CV MSE: ', mse_lr_cv.mean())
print('Poly Reg CV R2: ', r2_lr_cv.mean())

Poly Reg Train R2:  0.7822991655202127
Poly Reg CV MSE:  13.83954994256914
Poly Reg CV R2:  0.776794597461407


Again, not very impressive performance before finetuning

In [69]:
#the rho paramter is named l1_ratio in sklearn
param_grid = [{
    'alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1 , 5, 10, 50, 100],
    'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}]

grid_search = GridSearchCV(enet, param_grid, cv=5, scoring='r2', return_train_score=True)

grid_search.fit(trainX_poly2,trainY)

c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 943.9988561677949, tolerance: 1.5077993949579833
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 901.8842229289004, tolerance: 1.5381773487394963
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 830.5124412332281, tolerance: 1.3696142184873954
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 958.1739053909638, tolerance: 1.4991293221757322
  po

c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 497.342881151287, tolerance: 1.3696142184873954
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 461.9908627165962, tolerance: 1.4991293221757322
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 549.2123111908281, tolerance: 1.5106677573221763
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 456.8897536081696, tolerance: 1.5077993949579833
  pos

c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 41.69199023285978, tolerance: 1.3696142184873954
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 53.618959350570776, tolerance: 1.4991293221757322
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 97.73720495879434, tolerance: 1.5106677573221763
  positive)
c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 37.97421036592391, tolerance: 1.5077993949579833
  p

c:\python\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 694.053601255681, tolerance: 1.8574045234899328
  positive)


GridSearchCV(cv=5, error_score=nan,
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5,
                                    10, 50, 100],
                          'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                                       0.9]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='r2', verbose=0)

There are too many models to look at individually now, we will just focus on the best one

In [62]:
grid_search.best_params_

{'alpha': 0.005, 'l1_ratio': 0.2}

In [70]:
grid_search.best_score_

0.8507592522514302

Obtain the best Elastic-net model

In [63]:
best_enet = grid_search.best_estimator_

And test it on the testing data

In [64]:
best_enet.score(testX_poly2, testY)

0.8947727104157833

To sum up, the three best models' R2:

|Model|Training CV R2| Testing R2|
|-----|--------------|-----------|
|Ridge|0.852         |0.893      |
|LASSO|0.851         |0.895      |
|ENet |0.851         |0.895      |

Which are very similar in both training and testing data. Depending on the data, these models' performance may vary more, but you should not expect to see tremendous differences. After all, all three are different versions of linear regression models.